<a href="https://colab.research.google.com/github/braveenth/Canadian-Weather-AI/blob/main/Canadian_Weather_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Canadian Weather AI
By: Braveenth Rasanayagam

Multi-modal
- Vision
- Text to Voice

Cloud Run Jobs

In [27]:
image_url = "https://weather.gc.ca/data/jet_stream/tempmapwx_e.gif"
print(image_url)
!curl -o case_image.png {image_url}

from IPython.display import Image
Image("/content/jet_stream.png")
Image(url=image_url)

https://weather.gc.ca/data/jet_stream/tempmapwx_e.gif
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  275k  100  275k    0     0   730k      0 --:--:-- --:--:-- --:--:--  731k


In [28]:
import os

try:
    # Attempt to import the google.colab module to see if the program is running in Colab.
    from google.colab import userdata
    print("Running on Google Colab")
    !pip install openai
    !pip install pydub
    from openai import OpenAI
    client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
    !mkdir -p /content/text
    !mkdir -p /content/voice
    !mkdir -p /content/assets
    !curl -o "/content/assets/WeatherNetwork.mp3" "https://storage.googleapis.com/can-weather-ai/assets/WeatherNetwork.mp3"

except ImportError:
    # The ImportError exception will be raised if the google.colab module is not found,
    # indicating that the program is not running inside Google Colab.
    print("Running outside of Google Colab")
    client = OpenAI(api_key=(os.environ.get('OPENAI_API_KEY')))

from openai import OpenAI
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

Running on Google Colab
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 64.1M  100 64.1M    0     0  98.9M      0 --:--:-- --:--:-- --:--:-- 98.9M


In [29]:
canadianMetric = {}

canadianMetric[0] = "not Canadian"
canadianMetric[1] = "somewhat Canadian"
canadianMetric[2] = "slightly Canadian"
canadianMetric[3] = "moderately Canadian"
canadianMetric[4] = "very Canadian"
canadianMetric[5] = "as Canadian as possible"

In [30]:
gpt_mode = "gpt-4-vision-preview"
imagePrompt = "You are a Canadian weatherperson. Present this jet stream information to an audience. Make it sound very Canadian. It should sound as Canadian as absolutely possible."
image_url = "https://weather.gc.ca/data/jet_stream/tempmapwx_e.gif"

response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": imagePrompt},
        {
          "type": "image_url",
          "image_url": {
            "url": image_url,
          },
        },
      ],
    }
  ],
  max_tokens=600,
  temperature=0.7,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hey there, folks! Grab yourself a cup of Timmies and let's take a look at what's cookin' with our Canadian weather, eh? We're taking a gander at the jet stream map valid for Wednesday, March 20, 2024, at 18:00 UTC, brought to you by Environment Canada.\n\nAlrighty, we've got a classic Canadian setup here with the jet stream dippin' down and bringing in some chilly air from the north. You betcha, it's gonna be colder than a polar bear's toenails in some parts, so you'll want to keep your toque handy and maybe throw on an extra layer, like your favourite hockey jersey, before you head out for a rip.\n\nOver on the West Coast, British Columbia's lookin' like it's in for a mix of conditions, with the possibility of some snowfall up in the mountains, eh? Perfect for hittin' the slopes, just make sure you're prepared for that mountain weather.\n\nScooching across to the Prairies, we've got a high-pres

In [31]:
weatherScript = response.choices[0].message.content
print(weatherScript)

Hey there, folks! Grab yourself a cup of Timmies and let's take a look at what's cookin' with our Canadian weather, eh? We're taking a gander at the jet stream map valid for Wednesday, March 20, 2024, at 18:00 UTC, brought to you by Environment Canada.

Alrighty, we've got a classic Canadian setup here with the jet stream dippin' down and bringing in some chilly air from the north. You betcha, it's gonna be colder than a polar bear's toenails in some parts, so you'll want to keep your toque handy and maybe throw on an extra layer, like your favourite hockey jersey, before you head out for a rip.

Over on the West Coast, British Columbia's lookin' like it's in for a mix of conditions, with the possibility of some snowfall up in the mountains, eh? Perfect for hittin' the slopes, just make sure you're prepared for that mountain weather.

Scooching across to the Prairies, we've got a high-pressure system that's keepin' things clearer in Alberta and Saskatchewan, but don't let that fool ya 

In [32]:
# Save the weatherScript as a file
from datetime import datetime

date = datetime.now().strftime("%Y-%m-%d")
count = 0

while os.path.exists(f"/content/text/ai-canadian-jetstream-{date}_{count}.txt"):
    count += 1
file = open(f"/content/text/ai-canadian-jetstream-{date}_{count}.txt", "w")
file.write(weatherScript)
file.close()

In [33]:
from openai import OpenAI

try:
    # Attempt to import the google.colab module to see if the program is running in Colab.
    from google.colab import userdata
    print("Running on Google Colab")
    audioClient = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

except ImportError:
    # The ImportError exception will be raised if the google.colab module is not found,
    # indicating that the program is not running inside Google Colab.
    import os
    print("Running outside of Google Colab")
    audioClient = OpenAI(api_key=(os.environ.get('OPENAI_API_KEY')))

audioResponse = audioClient.audio.speech.create(
    model="tts-1",
    voice="onyx",
    input=weatherScript,
)

filename = f"weatherperson-output-{date}.mp3"

Running on Google Colab


In [34]:
audioResponse.stream_to_file(filename)

<ipython-input-34-02f298be9c94>:1: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  audioResponse.stream_to_file(filename)


In [36]:
from pydub import AudioSegment

# Export the result
weatherPersonFileCount = 0
while os.path.exists(f"/content/weatherperson-output-{date}_{weatherPersonFileCount}.wav"):
    weatherPersonFileCount += 1

weatherpersonFile = f"/content/{filename}-{count}"
audioResponse.stream_to_file(f"/content/weatherperson-output-{date}_{weatherPersonFileCount}.wav")

# Obtain Audio Files
audio_bg_music = AudioSegment.from_file("/content/assets/WeatherNetwork.mp3", format="mp3")
audio_weatherperson = AudioSegment.from_file(f"/content/weatherperson-output-{date}_{weatherPersonFileCount}.wav", format="mp3")

# BG Music Segments from https://www.youtube.com/watch?v=6E2uzYAGPQU
#####################################################################
# 01. Morning Report (1998) Conclusions 0:00. #
# 02. Autumn (1999) Flying V.1 1:59 #
# 03. Holidays (2000) Xmas Spirit 3:29
# 04. Biding My Time 4:59
# 05. Winter (2002-'03) Life to the Full 7:12
# 06. Spring-Autumn (2003) Healthy Outlook 8:42
# 07. Winter (2003-04) Embrace Life 10:43
# 08. Primary Theme (2004-05) Lazing on the Slopes 13:16
# 09. Theme (2005-06) Windstar 15:17
# 10. Theme (2006-10) 17:17
# 11. Theme (2010-201?) Song Contest Winner 19:37
# 12. Holiday Version of Theme 11 (2011) 21:58
# 13. 25th Anniversary (2004-05 Secondary Theme) 25:02

segments = [0] * 12
segments[0] = [0, 119000, 5]
segments[1] = [119000, 209000, 5]
segments[2] = [209000, 299000, 5]
segments[3] = [299000, 432000, 7]
segments[4] = [432000, 522000, 5]
segments[5] = [522000, 643000, 5]
segments[6] = [209000, 796000, 5]
segments[7] = [209000, 917000, 5]
segments[8] = [917000, 1037000, 5]
segments[9] = [1037000, 1177000, 5]
segments[10] = [1177000, 1318000, 5]
segments[11] = [1318000, 1502000, 5]

# Define start and end times in milliseconds
# Start at 17:22
# End at 17:3
start_time = 1042000  # Start at 10 seconds
end_time = start_time + 120000

# Use a Random number between 0 an 11 to obtain the segment number
import random
segment_number = random.randint(0, 11)

# Using Segments Here
start_time = segments[segment_number][0] + 1000
end_time = segments[segment_number][1]

# Slice the audio segment to the desired part
specific_part = audio_bg_music[start_time:end_time]

# Lower the volume by 10 dB
specific_part_quieter = specific_part - segments[segment_number][2]

# Overlay the audio files
# Here, overlay_audio will start at 0 milliseconds into base_audio
combined = specific_part_quieter.overlay(audio_weatherperson, position=0)

# Export the result
count = 0
while os.path.exists(f"/content/voice/ai-canadian-jetstream-{date}_{count}.mp3"):
    count += 1
combined.export(f"/content/voice/ai-canadian-jetstream-{date}_{count}.mp3", format="mp3")


<ipython-input-36-265742550ebd>:9: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  audioResponse.stream_to_file(f"/content/weatherperson-output-{date}_{weatherPersonFileCount}.wav")


<_io.BufferedRandom name='/content/voice/ai-canadian-jetstream-2024-03-20_0.mp3'>